In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
# first we're going to pull the raw images
repo = 'embargo'
instrument = 'LSSTComCam'
# DRP run containing processed images
collections = ['LSSTComCam/runs/DRP/20241101_20241211/w_2024_50/DM-48128']

butler = Butler(repo, collections=collections, instrument=instrument)

registry = butler.registry

In [ ]:
day = 20241207
datasetType = 'raw'
block = 'BLOCK-T332'
dims = ['exposure']
result = registry.queryDataIds(  datasets=datasetType, dimensions = dims,
          where = f"day_obs = {day} and exposure.science_program='{block}'"
                              )
results = [r for r in result]

print (f"got {len(results)} images")


In [ ]:
exp=""
exposures = []
for r in results:
   if r['exposure'] != exp:
        exp=  r['exposure']
        exposures.append(exp)

print (f"there are {len(exposures)} exposures")
print(exposures)
exposure_list = exposures

In [ ]:
# first we see what the raw images use
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[3]}')

images = []
for count, exp in enumerate(datasetRefs):
    #fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get(datasetType,exp.dataId)
    images.append(exp)

print(len(images))

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

for exp in images[:2]:
    dataId=exp.dataId
    img = butler.get('raw', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()

In [ ]:
# now we need to slightly modify to pull calexp from a different butler repo
butler = butlerUtils.makeDefaultButler("LSSTComCam")
registry = butler.registry


In [ ]:
day = 20241207
datasetType = 'postISRCCD'
dims = ['exposure']
result = registry.queryDataIds(  datasets=datasetType, dimensions = dims,
          where = f"day_obs = {day}"
                              )
results = [r for r in result]

print (f"got {len(results)} images")

In [ ]:
exp=""
match_exposures = []
for r in results:
   if r['exposure'] != exp:
        exp=  r['exposure']
        if exp in exposure_list:
            match_exposures.append(exp)

print (f"there are {len(match_exposures)} matched exposures")

In [ ]:
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[3]}')

In [ ]:
images = []
for count, exp in enumerate(datasetRefs):
    #fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get(datasetType,exp.dataId)
    images.append(exp)

print(len(images))

In [ ]:
for exp in images[:2]:
    dataId=exp.dataId
    img = butler.get('postISRCCD', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()